In [3]:
import numpy as np
from numpy import linalg as LA

import cvxpy as cp
import matplotlib.pyplot as plt

# import mosek

In [4]:
# Creating test case
# - Making sure SNR is always around 2.0
def gen_data(RANK = 5, N = 50, TRACE = 1, is_print = True):    
    M = N * 20 * RANK
    C = 0.5 / RANK

    V = np.random.normal(0, 1, N * RANK).reshape(N, RANK)
    Q, R = np.linalg.qr(V)
    print("Q", Q.shape, LA.norm(Q, 2))

    X_star = np.matmul(Q, np.transpose(Q)) / RANK * TRACE
    print("tr(X_star)", np.trace(X_star))

    A = np.zeros([M, N])
    B = np.zeros([M, N])
    Y_0 = np.zeros(M)

    for i in range(M):
        a = np.random.rand(1, N) - 0.5
        a_norm = LA.norm(a)
        a /= a_norm

        b = np.random.rand(N, 1) - 0.5
        b_norm = LA.norm(b)
        b /= b_norm

        y = np.matmul(a, X_star)
        y = np.matmul(y, b)

        A[i, :] = a
        B[i, :] = b.squeeze()
        Y_0[i] = y

    noise = np.random.normal(0, np.sqrt(C) / N * TRACE, M)
    Y = Y_0 + noise

    snr = (LA.norm(Y_0) / LA.norm(noise)) ** 2
    print("SNR", snr)

    if is_print:
        print(X_star)
    
    
    return X_star, A, B, Y

# Problem 1 -- Quadratic Measurement

$$ min_{X \ge 0, Tr(X) = \tau} f(X) = \frac{1}{2} \sum_{i = 1} ^ M (a_i^T X b_i - y_i)^2 $$

$$ \bigtriangledown f(X) = \sum_{i = 1} ^ M (a_i^T X b_i - y_i) (b_i a_i^T + a_i b_i^T) / 2 $$

In [8]:
def f(X, A, B, Y):
    return np.sum((np.sum(np.matmul(A, X) * B, 1) - Y) ** 2) / 2


def grad_f(X, A, B, Y):
    N = X.shape[0]
    M = A.shape[0]
    grad = np.zeros([N, N])
    for i in range(M):
        a = A[i, :].reshape(1, N)
        b = B[i, :].reshape(N, 1)
        y = Y[i]

        g = np.matmul(a, X)
        g = np.matmul(g, b)
        ab = np.matmul(b, a)
        grad += (g - y) * (ab + ab.transpose()) / 2

    return grad


def error(X_hat, X_star, scale = 1):
    return (LA.norm(X_hat * scale - X_star) / LA.norm(X_star)) ** 2

## Recovery with raw CVXPY

In [465]:
def cvx_solve(A, B, Y, X_star, tau):
    N = X_star.shape[0]
    X = cp.Variable((N, N), PSD=True)
    objective = cp.Minimize(cp.sum_squares(cp.sum(cp.multiply(A * X, B), 1) - Y) / 2)
    constraints = [X >> 0, cp.trace(X) == tau]
    prob = cp.Problem(objective, constraints)

    #prob.solve(solver=cp.MOSEK)
    f_hat = prob.solve()
    print("Solve time", prob.solver_stats.solve_time)

    X_hat = X.value
    recover_error = error(X_hat, X_star, N / tau)

    print("Optimal value", f_hat)
    print("Recover error", recover_error)
    print("tr(X_hat)", np.trace(X_hat))
    print("tr(X_star)", np.trace(X_star))
    
    return X_hat

In [479]:
for N in [30]:
    for RANK in [1, 5]:
        ## Generate data
        X_star, A, B, Y = gen_data(RANK, N, N, False)
        print()

        ## Solve
        tau = N / 2
        X_hat = cvx_solve(A, B, Y, X_star, tau)
        print()

        ## Eigen values
        print("-- Eigen values --")
        w, v = LA.eig(X_hat * N / tao)
        sorted_w = sorted(w.real)[::-1]
        print('X_hat', sorted_w)
        print()

        w, v = LA.eig(X_star)
        sorted_w = sorted(w.real)[::-1]
        print('X_star', sorted_w)
        print()

        ## Gap of nabla f
        print("-- Gap --")
        grad = grad_f(X_hat, A, B, Y)
        w, v = LA.eig(grad)
        sorted_w = sorted(w.real)
        gap = sorted_w[RANK] - sorted_w[RANK - 1]
        print(sorted_w)
        print("Gap", gap)
        print("--------------------------------------\n")

Q (30, 1) 1.0000000000000002
tr(X_star) 30.000000000000014
SNR 2.311763160778531

Solve time 884.191021
Optimal value 232.60911691534582
Recover error 0.041805941409745334
tr(X_hat) 15.00009212646629
tr(X_star) 30.000000000000014

-- Eigen values --
X_hat [30.000675117916664, 0.0016963628145736217, 0.0015014294308355932, 0.0013695366805570324, 0.0011555714223008828, 0.0011184790499234444, 0.0009449711628816513, 0.0008883056790835052, 0.0007656643333106696, 0.00045620691636686674, 0.0003236681305930777, 0.00028855862686799123, 0.00016562572186469782, 6.892715913062781e-05, -6.806673315069342e-05, -0.0001215639903070327, -0.0002481337201157985, -0.0002683388673448866, -0.0003322532485570707, -0.0003959221953679995, -0.000477590788329282, -0.0005743273987285195, -0.0006295597804574227, -0.0008335391604388587, -0.0009357288258981252, -0.0010214425777204774, -0.0011218720953307176, -0.001198039720772363, -0.0013085489072204863, -0.0016992441026240852]

X_star [30.000000000000014, 8.18140317

## Recovery with Algorithm 1 -- from Rank-1 paper

In [ ]:
def alg1_solve(A, B, Y, X_star, epoch = 100, eta = 0.05, print_epoch = -1):
    ## Initialization
    N = X_star.shape[0]
    X = np.random.rand(N, 1) - 0.5
    X = X / LA.norm(X)
    if print_epoch >= 0: 
        print("X", X.shape, LA.norm(X))
    X = np.matmul(X, X.transpose())
    if print_epoch >= 0: 
        print("tr(X)", np.trace(X))

    ## Run
    err_list = [error(X, X_star)]
    pred_list = [f(X, A, B, Y)]
    
    if print_epoch >= 0: 
        print("- Iterating started -")
    for i in range(epoch):
        if i > 0 and i % 300 == 0:
            eta /= 2
        grad = -grad_f(X, A, B, Y)
        eig, vec = LA.eig(grad)
        w = list(eig)
        ind = w.index(max(w))
        v = vec[:, ind].reshape(N, 1)
        vvT = np.matmul(v, v.transpose())
        X = (1 - eta) * X + eta * vvT
        
        err = error(X, X_star)
        pred = f(X, A, B, Y)
        err_list.append(err)
        pred_list.append(pred)
        if print_epoch < 0: 
            continue
        if print_epoch == 0 or i % print_epoch == 0:
            print(i, eta, err, pred)
        
    if print_epoch >= 0: 
        print("- Iterating finished - ")
        
    return err_list, pred_list, X

In [480]:
### solve 
X_star, A, B, Y = gen_data(5, 30, 1, False)
err_list, pred_list, X = alg1_solve(A, B, Y, X_star, 200, ETA, 10)
print("Recovery error", min(err_list))
print()

### analysis
print("-- Eigen values --")
w, v = LA.eig(X)
sorted_w = sorted(w.real)
print('X_hat', sorted_w)
print()

w, v = LA.eig(X_star)
sorted_w = sorted(w.real)
print('X_star', sorted_w)
print()

print("-- Gap --")
grad = grad_f(X, A, B, Y)
w, v = LA.eig(grad)
sorted_w = sorted(w.real)
gap = sorted_w[RANK] - sorted_w[RANK - 1]
print(sorted_w)
print(gap)

Q (30, 5) 1.0000000000000002
tr(X_star) 1.0000000000000002
SNR 2.2156178162758327
X (30, 1) 0.9999999999999999
tr(X) 1.0000000000000002
- Iterating started -
0 0.05 4.9311818571079815 1.7624883645788019
10 0.05 1.7510975999438503 0.6968382322128025
20 0.05 0.6207780722557897 0.33867422038823647
30 0.05 0.2276947262881886 0.21720575674916207
40 0.05 0.0969299151249563 0.1762774852030643
50 0.05 0.06404794519587388 0.16188391540325558
60 0.05 0.05274387631518484 0.15660582000592355
70 0.05 0.04458472447452509 0.1539152033558361
80 0.05 0.041302220864861364 0.15299071832942043
90 0.05 0.0422446257905123 0.1527094682844955
100 0.05 0.042198656306533064 0.15242577946803734
110 0.05 0.039193371957738185 0.1522711176069707
120 0.05 0.041742777513419656 0.15203888053989364
130 0.05 0.039800705099866655 0.15176276320613435
140 0.05 0.041783461755554085 0.15151406243394885
150 0.05 0.04066325828787662 0.15144803715831412
160 0.05 0.044530873956617215 0.1518168131567504
170 0.05 0.049289761256675

In [483]:
EPOCH = 500
ETA = 0.05

for N in [20, 50, 100]:
    for RANK in [1, 5]:
        ### solve 
        X_star, A, B, Y = gen_data(RANK, N, 1, False)
        err_list, pred_list, X = alg1_solve(A, B, Y, X_star, EPOCH, ETA)

        ### analysis
        grad = grad_f(X, A, B, Y)
        w, v = LA.eig(grad)
        sorted_w = sorted(w.real)
        gap = sorted_w[RANK] - sorted_w[RANK - 1]
        # print(sorted_w)
        # print("Gap", gap)
        print('Recover error', min(err_list))
        print('f', min(pred_list))
        print('Gap', gap)
        print()


        # x = range(EPOCH + 1)
        # plt.plot(x, err_list, label='algo1 N={} rank={}'.format(N, RANK))
        # plt.ylabel('error')
        # plt.xlabel('epoch')
        # plt.legend()
        # plt.ylim(0, 1)
        # plt.show()

Q (20, 1) 0.9999999999999998
tr(X_star) 0.9999999999999998
SNR 1.949067561345384
Recover error 0.06332229497627243
f 0.21430926038161113
Gap 0.005349880522302092

Q (20, 5) 1.0
tr(X_star) 0.9999999999999996
SNR 1.8295551080124062
Recover error 0.0355619049578001
f 0.24758379251552165
Gap 0.014121117625921296

Q (50, 1) 1.0
tr(X_star) 1.0
SNR 1.7078361805966582
Recover error 0.07832151714339144
f 0.09239131101822506
Gap 0.0046099006532408365

Q (50, 5) 1.0000000000000002
tr(X_star) 1.0
SNR 2.030583270321082
Recover error 0.04262229177248253
f 0.0918843759292294
Gap 0.0014746216823460848

Q (100, 1) 1.0000000000000002
tr(X_star) 1.0000000000000002
SNR 1.8467267522871642
Recover error 0.07497384031008268
f 0.04521492958193245
Gap 0.0012675563122862138

Q (100, 5) 1.0
tr(X_star) 1.0000000000000002
SNR 1.953743545084286
Recover error 0.05171792797503405
f 0.046821506101643234
Gap 0.00015629220776016556



## Recovery with Algorithm 2 -- from Lijun

In [48]:
def alg2_solve(A, B, Y, X_star, RANK, epoch = 100, eta = 0.05, print_epoch = -1):
    ## Initialization
    N = X_star.shape[0]
    X = np.random.rand(N, 1) - 0.5
    X = X / LA.norm(X)
    if print_epoch >= 0: 
        print("X", X.shape, LA.norm(X))
    X = np.matmul(X, X.transpose())
    if print_epoch >= 0: 
        print("tr(X)", np.trace(X))

    ## Run
    err_list = [error(X, X_star)]
    pred_list = [f(X, A, B, Y)]
    
    if print_epoch >= 0: 
        print("- Iterating started -")
    for i in range(epoch):
        if i > 0 and i % 300 == 0:
            eta /= 2
            
        # Compute V
        grad = -grad_f(X, A, B, Y)
        eig, vec = LA.eig(grad)
        idx = eig.argsort()[::-1][:RANK]
        vec = vec[:,idx]
        
        # Compute S
        S = cp.Variable((RANK, RANK), PSD=True)
        VSVT = vec * S * vec.transpose()
        new_X = (1 - eta) * X + eta * VSVT
        objective = cp.Minimize(cp.sum_squares(cp.sum(cp.multiply(A * new_X, B), 1) - Y) / 2)
        constraints = [S >> 0, cp.trace(S) == 1]
        prob = cp.Problem(objective, constraints)
        prob.solve()

        S_hat = S.value
        VSVT = np.matmul(vec, S_hat)
        VSVT = np.matmul(VSVT, vec.transpose())
        
        # Update
        X = (1 - eta) * X + eta * VSVT
        
        err = error(X, X_star)
        pred = f(X, A, B, Y)
        err_list.append(err)
        pred_list.append(pred)
        if print_epoch < 0: 
            continue
        if print_epoch == 0 or i % print_epoch == 0:
            print(i, eta, err, pred)
        
    if print_epoch >= 0: 
        print("- Iterating finished - ")
        
    return err_list, pred_list, X

In [45]:
### solve 
RANK = 5
X_star, A, B, Y = gen_data(RANK, 30, 1, False)
err_list, pred_list, X = alg2_solve(A, B, Y, X_star, RANK, 200, 0.05, 10)
print("Recovery error", min(err_list))
print()

### analysis
print("-- Eigen values --")
w, v = LA.eig(X)
sorted_w = sorted(w.real)
print('X_hat', sorted_w)
print()

w, v = LA.eig(X_star)
sorted_w = sorted(w.real)
print('X_star', sorted_w)
print()

print("-- Gap --")
grad = grad_f(X, A, B, Y)
w, v = LA.eig(grad)
sorted_w = sorted(w.real)
gap = sorted_w[RANK] - sorted_w[RANK - 1]
print(sorted_w)
print(gap)

Q (30, 5) 1.0000000000000002
tr(X_star) 0.9999999999999999
SNR 2.172562309092102
X (30, 1) 1.0
tr(X) 0.9999999999999999
- Iterating started -
0 0.05 5.355463015960123 2.030739568913372
10 0.05 1.9723904230146527 0.8136205895896613
20 0.05 0.7298758692624865 0.38955341351618533
30 0.05 0.27902474000690386 0.23980939263147827
40 0.05 0.12190937442836479 0.18598546725507753
50 0.05 0.06898666835552923 0.16621369967584348
60 0.05 0.05031104887168979 0.1585503719974074
70 0.05 0.04392459644653753 0.15540227287332864
80 0.05 0.04184635005182933 0.15403035020059336
90 0.05 0.04186594532448527 0.15338725921993324
100 0.05 0.04129484612141237 0.1530492605152008
110 0.05 0.04119160167684672 0.15287518711996337
120 0.05 0.041032063777048336 0.15278005759598445
130 0.05 0.040882001376993625 0.15271933893656642
140 0.05 0.041873847830765795 0.1526992720824933
150 0.05 0.04126717340877165 0.15266976698864493
160 0.05 0.04088823106297907 0.1526486006592886
170 0.05 0.040785771708819626 0.152638988439

In [ ]:
EPOCH = 100
ETA = 0.05

for N in [20, 50, 100]:
    for RANK in [5]:
        ### solve 
        X_star, A, B, Y = gen_data(RANK, N, 1, False)
        err_list, pred_list, X = alg2_solve(A, B, Y, X_star, RANK, EPOCH, ETA, 20)

        ### analysis
        grad = grad_f(X, A, B, Y)
        w, v = LA.eig(grad)
        sorted_w = sorted(w.real)
        gap = sorted_w[RANK] - sorted_w[RANK - 1]
        # print(sorted_w)
        # print("Gap", gap)
        print('Recover error', min(err_list))
        print('f', min(pred_list))
        print('Gap', gap)
        print()

Q (20, 5) 1.0000000000000002
tr(X_star) 0.9999999999999998
SNR 1.920102932821499
X (20, 1) 1.0
tr(X) 1.0000000000000004
- Iterating started -
0 0.05 5.1230372610533 2.5398857473831598
20 0.05 0.661521589398847 0.5070286933800487
40 0.05 0.10550057478156903 0.2740290545174021
60 0.05 0.04145913023881342 0.24600067308011542
80 0.05 0.03367852032281308 0.24215980381189928
- Iterating finished - 
Recover error 0.03186568045195402
f 0.24150925176622895
Gap 0.019799704176910814

Q (50, 5) 1.0
tr(X_star) 1.0
SNR 2.0608451562341883
X (50, 1) 1.0
tr(X) 1.0
- Iterating started -
0 0.05 5.044813235028795 1.0980399512261108
20 0.05 0.645797598298581 0.21372198511143842
40 0.05 0.1077162072542343 0.11038153196460328
60 0.05 0.048059257678714266 0.09668429273155621
80 0.05 0.03984720808094076 0.09421743906988482
- Iterating finished - 
Recover error 0.03852744159834524
f 0.09363111603937857
Gap 0.004388179982397843

Q (100, 5) 1.0000000000000002
tr(X_star) 1.0
SNR 1.962133651014831
X (100, 1) 1.0
tr

In [1]:
91668522 Jan 19 19:57 train-all-shuf.lst
-rw-rw-r-- 1 qiantong qiantong    8716148 Jan 24 12:57 train-clean-100.lst
-rw-rw-r-- 1 qiantong qiantong   31721194 Jan 24 12:57 train-clean-360.lst
-rw-rw-r-- 1 qiantong qiantong   43356432 Jan 24 12:57 train-other-500.lst

SyntaxError: invalid syntax (<ipython-input-1-2bd8bf95fafc>, line 1)

In [2]:
print(8716148 + 31721194 + 43356432)

83793774
